<a href="https://colab.research.google.com/github/sravya19path/My-Projects---Data-Analytics/blob/master/Original_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IDS 561 Analytics for Big Data

## Homework 1

## Due date: 6:00 pm 10/09/2019

In this homework you will mimic the process of MapReduce task. Specifically, you will write your own map and reduce functions (without distributing to several machines) to mimic the process of mapper and reducer. 


The task is to count the number of occurrences
of each word in a text file. This program, known as Word Count, is the equivalent of the standard
Hello, world! program you typically write when you learn a new programming language.
 

While doing this homework you will learn:

1) how to prepare the data. 

2) how to write map and reduce functions.

3) get a better understanding of how mapper and reducer work.

The input of this homework is a text document which includes several paragraphs. You can download it from here: http://www.gutenberg.org/cache/epub/100/pg100.txt

It is a raw data. You need to do some data cleaning works to prepare it for next step. 

Fill in the cells that have "TO DO", and upload your **.ipynb document** on Blackboard, naming it with "your NetID_HW1", e.g., "tsmith7_HW1".



**NOTE: A toy example is included here to give a basic idea about the task. To finish the homework, you should:**

**1. Download the text file from the link above and use it for the homework.**

**2. Delete the example code and write your own code under "#TO DO".**

**3. The final output format can be different from the toy example as long as the word count result is shown clearly.**

In [0]:
# Importing required packages
import string # to remove punctuation
import re
import pandas as pd
import itertools

##  Step 1.   Preparing data

In [2]:
# Importing the text file
from google.colab import files
uploaded = files.upload()

Saving pg100.txt to pg100.txt


In [6]:
# Opening and Printing 5000 lines of the text file
text = open('pg100.txt', encoding='utf-8-sig')
text1 = text.read()
print(text1[0:5000])

The Project Gutenberg EBook of The Complete Works of William Shakespeare, by
William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org

** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **
**     Please follow the copyright guidelines in this file.     **

Title: The Complete Works of William Shakespeare

Author: William Shakespeare

Posting Date: September 1, 2011 [EBook #100]
Release Date: January, 1994

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK COMPLETE WORKS--WILLIAM SHAKESPEARE ***




Produced by World Library, Inc., from their Library of the Future




This is the 100th Etext file presented by Project Gutenberg, and
is presented in cooperation with World Library, Inc., from their
Library of the Future and Shakespeare CDROMS.  Project Gut

In [7]:
# Deleting all the numbers and @ in the text file
# printing first 5000 lines

text2 = re.sub(r'\S*@\S*\s?', '', text1)
text3 = ''.join([i for i in text2 if not i.isdigit()])
print(text3[0:5000])

The Project Gutenberg EBook of The Complete Works of William Shakespeare, by
William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org

** This is a COPYRIGHTED Project Gutenberg eBook, Details Below **
**     Please follow the copyright guidelines in this file.     **

Title: The Complete Works of William Shakespeare

Author: William Shakespeare

Posting Date: September ,  [EBook #]
Release Date: January, 

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK COMPLETE WORKS--WILLIAM SHAKESPEARE ***




Produced by World Library, Inc., from their Library of the Future




This is the th Etext file presented by Project Gutenberg, and
is presented in cooperation with World Library, Inc., from their
Library of the Future and Shakespeare CDROMS.  Project Gutenberg
often re

In [8]:
# Deleting all the punctuation marks
text4 = text3.translate(str.maketrans('','',string.punctuation))
print(text4[0:5000])

The Project Gutenberg EBook of The Complete Works of William Shakespeare by
William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever  You may copy it give it away or
reuse it under the terms of the Project Gutenberg License included
with this eBook or online at wwwgutenbergorg

 This is a COPYRIGHTED Project Gutenberg eBook Details Below 
     Please follow the copyright guidelines in this file     

Title The Complete Works of William Shakespeare

Author William Shakespeare

Posting Date September   EBook 
Release Date January 

Language English


 START OF THIS PROJECT GUTENBERG EBOOK COMPLETE WORKSWILLIAM SHAKESPEARE 




Produced by World Library Inc from their Library of the Future




This is the th Etext file presented by Project Gutenberg and
is presented in cooperation with World Library Inc from their
Library of the Future and Shakespeare CDROMS  Project Gutenberg
often releases Etexts that are NOT placed in the P

In [9]:
#Converting to lower case
text5 = text4.lower()

# Seperating each word with a space 
#Saving the words to a list named "data"
data = text5.split()  

#Printing the splitted list of 100 words
print(data[0:100])

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'by', 'william', 'shakespeare', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 'reuse', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'wwwgutenbergorg', 'this', 'is', 'a', 'copyrighted', 'project', 'gutenberg', 'ebook', 'details', 'below', 'please', 'follow', 'the', 'copyright', 'guidelines', 'in', 'this', 'file', 'title', 'the', 'complete', 'works', 'of', 'william', 'shakespeare', 'author', 'william', 'shakespeare', 'posting', 'date', 'september', 'ebook', 'release', 'date', 'january', 'language', 'english', 'start', 'of', 'this', 'project', 'gutenberg', 'ebook']


In [11]:
#Printing number of words obtained after data cleaning
print(len(data))

902512


##  Step 2.   Building Map function

In this part, you will produce a set of key-value pairs and collect all pairs with same key.

The input here is the "data" list you created above.

### Create key-value pairs

In [12]:
# Adding value 1 to each word in data list
data_pairs = []
for item in data:
  temp = item +' ' + str(1)
  data_pairs.append(temp)
  
#Printing 100 words
print(data_pairs[0:100])

['the 1', 'project 1', 'gutenberg 1', 'ebook 1', 'of 1', 'the 1', 'complete 1', 'works 1', 'of 1', 'william 1', 'shakespeare 1', 'by 1', 'william 1', 'shakespeare 1', 'this 1', 'ebook 1', 'is 1', 'for 1', 'the 1', 'use 1', 'of 1', 'anyone 1', 'anywhere 1', 'at 1', 'no 1', 'cost 1', 'and 1', 'with 1', 'almost 1', 'no 1', 'restrictions 1', 'whatsoever 1', 'you 1', 'may 1', 'copy 1', 'it 1', 'give 1', 'it 1', 'away 1', 'or 1', 'reuse 1', 'it 1', 'under 1', 'the 1', 'terms 1', 'of 1', 'the 1', 'project 1', 'gutenberg 1', 'license 1', 'included 1', 'with 1', 'this 1', 'ebook 1', 'or 1', 'online 1', 'at 1', 'wwwgutenbergorg 1', 'this 1', 'is 1', 'a 1', 'copyrighted 1', 'project 1', 'gutenberg 1', 'ebook 1', 'details 1', 'below 1', 'please 1', 'follow 1', 'the 1', 'copyright 1', 'guidelines 1', 'in 1', 'this 1', 'file 1', 'title 1', 'the 1', 'complete 1', 'works 1', 'of 1', 'william 1', 'shakespeare 1', 'author 1', 'william 1', 'shakespeare 1', 'posting 1', 'date 1', 'september 1', 'ebook 1',

### Sorting by key

In [14]:
# Grouping sublists with the same keys together
data_pairs.sort()

#Printing the lists
print(data_pairs[0:500])

['a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1', 'a 1'

Now you have finished the mapper part, good job!

##  Step 3.   Building Reduce function

In this part, you will collect all the values with same key, count the number and produce an output.

The input here is the "data" list you created in the second step.

In [18]:
#Avoiding repetition of words
remove_data_pairs = list(dict.fromkeys(data_pairs))
print(remove_data_pairs)
print(len(remove_data_pairs))

['a 1', 'aaron 1', 'aarons 1', 'abaissiez 1', 'abandon 1', 'abandond 1', 'abandoned 1', 'abase 1', 'abashd 1', 'abate 1', 'abated 1', 'abatement 1', 'abatements 1', 'abates 1', 'abatfowling 1', 'abbess 1', 'abbey 1', 'abbeys 1', 'abbominable 1', 'abbot 1', 'abbots 1', 'abbreviated 1', 'abed 1', 'abel 1', 'abels 1', 'abergany 1', 'abergavenny 1', 'abet 1', 'abetting 1', 'abhominable 1', 'abhor 1', 'abhorrd 1', 'abhorrdst 1', 'abhorred 1', 'abhorring 1', 'abhors 1', 'abhorson 1', 'abide 1', 'abides 1', 'abilities 1', 'ability 1', 'abilitys 1', 'abirding 1', 'abject 1', 'abjectly 1', 'abjects 1', 'abjurd 1', 'abjure 1', 'able 1', 'ableeding 1', 'abler 1', 'aboard 1', 'abode 1', 'aboded 1', 'abodements 1', 'aboding 1', 'abominable 1', 'abominably 1', 'abominations 1', 'abortive 1', 'abortives 1', 'abound 1', 'abounding 1', 'aboundst 1', 'about 1', 'above 1', 'abr 1', 'abraham 1', 'abrahams 1', 'abram 1', 'abreast 1', 'abreeding 1', 'abrewing 1', 'abridgd 1', 'abridge 1', 'abridged 1', 'abr

In [0]:
#Counting the sum value of pairs with same keys
for item in data_pairs_red:
  count = data_pairs.count(item)
  print("("+ item.split(' ')[0] + ")" + ":" + str(count))

(a):14668
(aaron):96
(aarons):1
(abaissiez):1
(abandon):4
(abandond):6
(abandoned):2
(abase):2
(abashd):1
(abate):14
(abated):3
(abatement):3
(abatements):1
(abates):1
(abatfowling):1
(abbess):25
(abbey):20
(abbeys):1
(abbominable):1
(abbot):9
(abbots):1
(abbreviated):1
(abed):9
(abel):1
(abels):1
(abergany):2
(abergavenny):8
(abet):1
(abetting):1
(abhominable):1
(abhor):19
(abhorrd):11
(abhorrdst):1
(abhorred):9
(abhorring):2
(abhors):4
(abhorson):21
(abide):38
(abides):8
(abilities):5
(ability):9
(abilitys):1
(abirding):4
(abject):11
(abjectly):1
(abjects):2
(abjurd):2
(abjure):4
(able):59
(ableeding):2
(abler):1
(aboard):28
(abode):9
(aboded):1
(abodements):1
(aboding):1
(abominable):14
(abominably):1
(abominations):1
(abortive):4
(abortives):1
(abound):7
(abounding):1
(aboundst):1
(about):408
(above):147
(abr):5
(abraham):3
(abrahams):1
(abram):5
(abreast):3
(abreeding):1
(abrewing):1
(abridgd):1
(abridge):2
(abridged):1
(abridgment):4
(abroach):3
(abroad):63
(abroadanon):1
(abroga